# 📌 Configurar Instancias y Balanceo de Carga en Google Cloud

## Introducción

Este laboratorio te guiará en la creación y configuración de múltiples máquinas virtuales (VM) en **Google Cloud**, instalando servidores web **Apache**, configurando reglas de firewall, y estableciendo un **Load Balancer** (balanceador de carga) para distribuir tráfico entre las instancias. Aprenderás a implementar un **Network Load Balancer** (capa 4) y un **Application Load Balancer** (capa 7), asegurando alta disponibilidad y escalabilidad para aplicaciones web. Este enfoque es ideal para empresas que necesitan garantizar que sus aplicaciones, como sistemas de contabilidad o portales de clientes, sean accesibles y eficientes.

💡 **Beneficio empresarial**: Los balanceadores de carga optimizan el rendimiento de las aplicaciones al distribuir el tráfico, reduciendo tiempos de inactividad y mejorando la experiencia del usuario, lo que es crítico para procesos empresariales como la gestión de transacciones o servicios al cliente.

Para más información, consulta la [Documentación oficial de Google Cloud Load Balancing](https://cloud.google.com/load-balancing/docs).

## 🚀 Objetivos del laboratorio

- Configurar regiones y zonas por defecto para gestionar recursos en **Google Cloud**.
- Crear múltiples máquinas virtuales con servidores web **Apache**.
- Establecer reglas de firewall para permitir tráfico HTTP.
- Configurar un **Network Load Balancer** para distribuir tráfico entre instancias.
- Implementar un **Application Load Balancer** para enrutamiento avanzado a nivel HTTP.
- Probar la distribución de tráfico para verificar alta disponibilidad.

💡 **Contexto empresarial**: Estas habilidades permiten a las empresas implementar aplicaciones escalables y confiables, como sistemas de facturación en línea, garantizando continuidad del negocio y satisfacción del cliente.

## 🚀 Tarea 1: Configurar la Región y Zona por Defecto

**Google Cloud** organiza sus recursos en **regiones** (ubicaciones geográficas) y **zonas** (subdivisiones dentro de una región). Configurar valores predeterminados simplifica la gestión de recursos.

### Establecer la región

In [ ]:
gcloud config set compute/region us-east1

**Explicación del comando**:  
- `gcloud config set compute/region`: Define la región predeterminada.  
- `us-east1`: Ejemplo de región; cámbialo según las instrucciones del laboratorio.  
💡 **Contexto empresarial**: Elegir una región cercana a los usuarios reduce la latencia, mejorando el acceso a aplicaciones como sistemas de gestión financiera.

### Establecer la zona

In [ ]:
gcloud config set compute/zone us-east1-b

**Explicación del comando**:  
- `gcloud config set compute/zone`: Define la zona predeterminada.  
- `us-east1-b`: Ejemplo de zona; cámbialo según las instrucciones.  
💡 **Contexto empresarial**: Configurar una zona asegura que los recursos, como servidores para aplicaciones contables, estén organizados y optimizados.

Para más información, consulta la [Documentación de regiones y zonas](https://cloud.google.com/compute/docs/regions-zones).

## 🚀 Tarea 2: Crear Múltiples Instancias de Servidores Web

Crearás tres máquinas virtuales (`www1`, `www2`, `www3`) con **Apache** instalado y una página web sencilla para identificar cada instancia. También configurarás una regla de firewall para permitir tráfico HTTP.

### Crear la primera VM (`www1`)

In [ ]:
gcloud compute instances create www1 \
  --zone=us-east1-b \
  --tags=network-lb-tag \
  --machine-type=e2-small \
  --image-family=debian-11 \
  --image-project=debian-cloud \
  --metadata=startup-script='#!/bin/bash
    apt-get update
    apt-get install apache2 -y
    service apache2 restart
    echo "<h3>Web Server: www1</h3>" | tee /var/www/html/index.html'

**Explicación del comando**:  
- `gcloud compute instances create www1`: Crea una VM llamada `www1`.  
- `--zone=us-east1-b`: Especifica la zona.  
- `--tags=network-lb-tag`: Etiqueta la VM para aplicar reglas de firewall.  
- `--machine-type=e2-small`: Define un tipo de máquina con recursos moderados.  
- `--image-family=debian-11 --image-project=debian-cloud`: Usa Debian 11 como sistema operativo.  
- `--metadata=startup-script`: Ejecuta un script al iniciar la VM para instalar **Apache** y crear una página web con el texto "Web Server: www1".  

### Crear la segunda VM (`www2`)

In [ ]:
gcloud compute instances create www2 \
  --zone=us-east1-b \
  --tags=network-lb-tag \
  --machine-type=e2-small \
  --image-family=debian-11 \
  --image-project=debian-cloud \
  --metadata=startup-script='#!/bin/bash
    apt-get update
    apt-get install apache2 -y
    service apache2 restart
    echo "<h3>Web Server: www2</h3>" | tee /var/www/html/index.html'

**Explicación del comando**: Similar al anterior, pero crea una VM llamada `www2` con una página que muestra "Web Server: www2".  

### Crear la tercera VM (`www3`)

In [ ]:
gcloud compute instances create www3 \
  --zone=us-east1-b \
  --tags=network-lb-tag \
  --machine-type=e2-small \
  --image-family=debian-11 \
  --image-project=debian-cloud \
  --metadata=startup-script='#!/bin/bash
    apt-get update
    apt-get install apache2 -y
    service apache2 restart
    echo "<h3>Web Server: www3</h3>" | tee /var/www/html/index.html'

**Explicación del comando**: Crea una VM llamada `www3` con una página que muestra "Web Server: www3".  
💡 **Contexto empresarial**: Múltiples VMs con servidores web aseguran redundancia, ideal para aplicaciones empresariales críticas, como portales de clientes.

### Crear regla de firewall para tráfico HTTP

In [ ]:
gcloud compute firewall-rules create www-firewall-network-lb \
  --target-tags network-lb-tag \
  --allow tcp:80

**Explicación del comando**:  
- `gcloud compute firewall-rules create`: Crea una regla de firewall.  
- `www-firewall-network-lb`: Nombre de la regla.  
- `--target-tags network-lb-tag`: Aplica la regla a VMs con la etiqueta `network-lb-tag`.  
- `--allow tcp:80`: Permite tráfico HTTP en el puerto 80.  

### Verificar las instancias

In [ ]:
gcloud compute instances list

**Explicación del comando**:  
- `gcloud compute instances list`: Lista las VMs creadas, mostrando sus IPs externas.  

### Probar las instancias

In [ ]:
curl http://[IP_ADDRESS]

**Explicación del comando**:  
- `curl http://[IP_ADDRESS]`: Prueba cada VM reemplazando `[IP_ADDRESS]` con la IP externa de `www1`, `www2` o `www3`. Debería mostrar el contenido de la página web (ej., "Web Server: www1").  
💡 **Contexto empresarial**: Verificar las instancias asegura que los servidores web estén listos para servir aplicaciones, como sistemas de facturación en línea.

Para más información, consulta la [Documentación de Compute Engine](https://cloud.google.com/compute/docs).

## 🚀 Tarea 3: Configurar un Network Load Balancer

Un **Network Load Balancer** (capa 4) distribuye el tráfico entre varias VMs para evitar saturación y mejorar la disponibilidad.

### Crear una dirección IP estática

In [ ]:
gcloud compute addresses create network-lb-ip-1 \
  --region us-east1

**Explicación del comando**:  
- `gcloud compute addresses create`: Reserva una IP estática para el balanceador.  
- `--region us-east1`: Define la región.  

### Crear un chequeo de salud

In [ ]:
gcloud compute http-health-checks create basic-check

**Explicación del comando**:  
- `gcloud compute http-health-checks create`: Crea un chequeo de salud para verificar que las VMs estén funcionando.  

### Crear un grupo de destino

In [ ]:
gcloud compute target-pools create www-pool \
  --region us-east1 \
  --http-health-check basic-check

**Explicación del comando**:  
- `gcloud compute target-pools create`: Crea un grupo de destino para las VMs.  
- `--http-health-check basic-check`: Asocia el chequeo de salud.  

### Añadir instancias al grupo

In [ ]:
gcloud compute target-pools add-instances www-pool \
  --instances www1,www2,www3

**Explicación del comando**:  
- `gcloud compute target-pools add-instances`: Añade las VMs `www1`, `www2` y `www3` al grupo.  

### Crear una regla de reenvío

In [ ]:
gcloud compute forwarding-rules create www-rule \
  --region us-east1 \
  --ports 80 \
  --address network-lb-ip-1 \
  --target-pool www-pool

**Explicación del comando**:  
- `gcloud compute forwarding-rules create`: Crea una regla para dirigir el tráfico a la IP estática en el puerto 80 al grupo de destino.  
💡 **Contexto empresarial**: Un **Network Load Balancer** garantiza que las aplicaciones empresariales, como portales web, estén disponibles incluso durante picos de tráfico.

Para más información, consulta la [Documentación de Network Load Balancing](https://cloud.google.com/load-balancing/docs/network).

## 🚀 Tarea 4: Enviar Tráfico al Network Load Balancer

Probarás cómo el balanceador distribuye el tráfico entre las VMs.

### Obtener la IP del balanceador

In [ ]:
gcloud compute forwarding-rules describe www-rule --region us-east1

**Explicación del comando**:  
- `gcloud compute forwarding-rules describe`: Muestra detalles de la regla de reenvío, incluyendo la IP estática.  

### Guardar la IP en una variable

In [ ]:
IPADDRESS=$(gcloud compute forwarding-rules describe www-rule --region us-east1 --format="json" | jq -r .IPAddress)

**Explicación del comando**:  
- `IPADDRESS=$(...)`: Almacena la IP del balanceador en una variable.  
- `jq -r .IPAddress`: Extrae la IP del resultado JSON.  

### Mostrar la IP

In [ ]:
echo $IPADDRESS

**Explicación del comando**:  
- `echo $IPADDRESS`: Muestra la IP almacenada.  

### Probar el balanceador

In [ ]:
while true; do curl -m1 $IPADDRESS; done

**Explicación del comando**:  
- `while true; do curl -m1 $IPADDRESS; done`: Envía solicitudes repetidas al balanceador, mostrando páginas alternadas de `www1`, `www2` o `www3`.  
- Presiona **Ctrl + C** para detener.  
💡 **Contexto empresarial**: Probar el balanceador asegura que el tráfico se distribuya correctamente, manteniendo la disponibilidad de aplicaciones críticas.

Para más información, consulta la [Documentación de Network Load Balancing](https://cloud.google.com/load-balancing/docs/network).

## 🚀 Tarea 5: Crear un Application Load Balancer

Un **Application Load Balancer** (capa 7) permite enrutamiento avanzado basado en HTTP/HTTPS, ideal para aplicaciones web complejas.

### Crear una plantilla de instancias

In [ ]:
gcloud compute instance-templates create lb-backend-template \
  --region=us-east1 \
  --network=default \
  --subnet=default \
  --tags=allow-health-check \
  --machine-type=e2-medium \
  --image-family=debian-11 \
  --image-project=debian-cloud \
  --metadata=startup-script='#!/bin/bash
    apt-get update
    apt-get install apache2 -y
    a2ensite default-ssl
    a2enmod ssl
    vm_hostname="$(curl -H "Metadata-Flavor:Google" \
    http://169.254.169.254/computeMetadata/v1/instance/name)"
    echo "Page served from: $vm_hostname" | \
    tee /var/www/html/index.html
    systemctl restart apache2'

**Explicación del comando**:  
- `gcloud compute instance-templates create`: Crea una plantilla para VMs con **Apache** configurado.  
- `--tags=allow-health-check`: Etiqueta para chequeos de salud.  
- `--metadata=startup-script`: Instala **Apache**, habilita SSL y muestra el nombre de la instancia en la página web.  

### Crear un grupo de instancias gestionado

In [ ]:
gcloud compute instance-groups managed create lb-backend-group \
  --template=lb-backend-template \
  --size=2 \
  --zone=us-east1-b

**Explicación del comando**:  
- `gcloud compute instance-groups managed create`: Crea un grupo gestionado con 2 instancias basadas en la plantilla.  
- `--size=2`: Define el número inicial de instancias.  

### Crear regla de firewall para chequeos de salud

In [ ]:
gcloud compute firewall-rules create fw-allow-health-check \
  --network=default \
  --action=allow \
  --direction=ingress \
  --source-ranges=130.211.0.0/22,35.191.0.0/16 \
  --target-tags=allow-health-check \
  --rules=tcp:80

**Explicación del comando**:  
- `gcloud compute firewall-rules create`: Crea una regla para permitir chequeos de salud desde las IPs de Google Cloud (`130.211.0.0/22`, `35.191.0.0/16`).  

### Crear chequeo de salud HTTP

In [ ]:
gcloud compute health-checks create http http-basic-check \
  --port 80

**Explicación del comando**:  
- `gcloud compute health-checks create http`: Crea un chequeo de salud para tráfico HTTP en el puerto 80.  

### Crear un servicio backend

In [ ]:
gcloud compute backend-services create web-backend-service \
  --protocol=HTTP \
  --port-name=http \
  --health-checks=http-basic-check \
  --global

**Explicación del comando**:  
- `gcloud compute backend-services create`: Crea un servicio backend para el balanceador.  
- `--protocol=HTTP`: Especifica el protocolo HTTP.  

### Añadir el grupo gestionado al servicio backend

In [ ]:
gcloud compute backend-services add-backend web-backend-service \
  --instance-group=lb-backend-group \
  --instance-group-zone=us-east1-b \
  --global

**Explicación del comando**:  
- `gcloud compute backend-services add-backend`: Añade el grupo gestionado al servicio backend.  

### Crear un mapa de URL

In [ ]:
gcloud compute url-maps create web-map-http \
  --default-service web-backend-service

**Explicación del comando**:  
- `gcloud compute url-maps create`: Crea un mapa de URL para enrutar tráfico al servicio backend.  

### Crear un proxy HTTP

In [ ]:
gcloud compute target-http-proxies create http-lb-proxy \
  --url-map web-map-http

**Explicación del comando**:  
- `gcloud compute target-http-proxies create`: Crea un proxy para manejar tráfico HTTP.  

### Crear una regla de reenvío global

In [ ]:
gcloud compute forwarding-rules create http-content-rule \
  --address=lb-ipv4-1 \
  --global \
  --target-http-proxy=http-lb-proxy \
  --ports=80

**Explicación del comando**:  
- `gcloud compute forwarding-rules create`: Dirige el tráfico a la IP estática global al proxy HTTP.  
💡 **Contexto empresarial**: Un **Application Load Balancer** permite enrutamiento avanzado, ideal para aplicaciones globales como portales de comercio electrónico.

Para más información, consulta la [Documentación de HTTP Load Balancing](https://cloud.google.com/load-balancing/docs/https).

## 🚀 Tarea 6: Probar el Application Load Balancer

Verificarás que el balanceador de aplicaciones distribuye el tráfico correctamente.

### Pasos
1. En **Google Cloud Console**, ve a **Load balancing** y selecciona el balanceador `web-map-http`.  
2. En la sección **Backend**, verifica que las instancias estén marcadas como **Healthy**.  
3. Accede a la IP del balanceador en un navegador con `http://[IP_ADDRESS]/`.  
4. Deberías ver una página con el texto "Page served from: [vm_hostname]", mostrando el nombre de la instancia que respondió.  

💡 **Contexto empresarial**: Probar el balanceador asegura que las aplicaciones empresariales, como sistemas de gestión de clientes, sean accesibles globalmente con alta disponibilidad.

Para más información, consulta la [Documentación de HTTP Load Balancing](https://cloud.google.com/load-balancing/docs/https).

## 🚀 Cuadro Resumen

| **Tarea** | **Descripción** | **Comandos Clave** |
|-----------|-----------------|-------------------|
| **1. Configurar región y zona** | Define región y zona por defecto para gestionar recursos. | `gcloud config set compute/region us-east1`<br>`gcloud config set compute/zone us-east1-b` |
| **2. Crear VMs con Apache** | Crea tres VMs con **Apache** y reglas de firewall para tráfico HTTP. | `gcloud compute instances create www1 ...`<br>`gcloud compute firewall-rules create ...` |
| **3. Configurar Network Load Balancer** | Reserva IP estática, crea grupo de destino y reglas para distribuir tráfico. | `gcloud compute addresses create ...`<br>`gcloud compute target-pools create ...` |
| **4. Enviar tráfico a Network Load Balancer** | Prueba la distribución de tráfico con `curl`. | `gcloud compute forwarding-rules describe ...`<br>`curl http://$IPADDRESS` |
| **5. Crear Application Load Balancer** | Configura balanceo avanzado con grupos gestionados, chequeos de salud y mapas de URL. | `gcloud compute instance-templates create ...`<br>`gcloud compute backend-services create ...` |
| **6. Probar Application Load Balancer** | Verifica que las VMs estén saludables y accede a la IP en un navegador. | |

💡 **Contexto empresarial**: Este resumen proporciona una guía clara para implementar balanceadores de carga, optimizando la disponibilidad y escalabilidad de aplicaciones empresariales.

## 🚀 Conclusión

Este laboratorio te ha permitido configurar múltiples máquinas virtuales con **Apache**, establecer reglas de firewall, y crear un **Network Load Balancer** y un **Application Load Balancer** en **Google Cloud**. Estas habilidades son esenciales para garantizar que las aplicaciones empresariales, como portales de clientes o sistemas de contabilidad, sean escalables, confiables y accesibles globalmente.

💡 **Beneficio empresarial**: Los balanceadores de carga reducen tiempos de inactividad, optimizan el uso de recursos y mejoran la experiencia del usuario, lo que es crítico para mantener la continuidad de negocio en entornos empresariales.